In [25]:
import os
import numpy as np
import torch as t
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

class Config():

    def __init__(self):

        self.device = 'cpu'
        self.seed = 0

        self.datasets_dir = ''

        self.n_epoch = 100
        self.batch_size = 10000
        self.n_worker = 0
        self.lr = 0.1

        self.ckpts_dir = ''

        return

class Task(Dataset):
    def __init__(self, cfg):
        self.data_ls = []
        with open(os.path.join('data/', 'test.csv'), 'r') as f:
            i=0
            for line in f.readlines():
                if i>0:
                    value_ls = line.strip('\n').split(',')
                    value_ls[3] = value_ls[3][9]
                    value_ls[4] = value_ls[4][9]
                    value_ls[1] = 0
                    for j in range(1,len(value_ls)):
                        if value_ls[j] == '':
                            value_ls[j] = 0.0
                        else:
                            value_ls[j] = float(value_ls[j])
#                     value_ls.pop(0)        
                    arr = np.array(value_ls)
                    arr = arr.astype(np.single)
                    self.data_ls.append(arr)
                i=i+1
        self.data = np.stack(self.data_ls,axis=0)
        return
    
    def __getitem__(self, idx):
        x = self.data[idx, 1:]
#         print(x)
        y = self.data[idx, 0]
        return x, y
        
    def __len__(self):
        return self.data.shape[0]
    
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.C0 = nn.Sequential(
            nn.Linear(24, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,1)
            )
        return
    
    def forward(self,x):
        pred_y = self.C0(x)
        return pred_y
    
with t.no_grad():
    with open('submission.csv', 'w') as f:    
        f.write('id,failure\n')
        
        model_ckpt_path = 'ckpts/model_e38.ckpt'
        
        cfg = Config()  
        t.backends.cudnn.deterministic = True
        t.backends.cudnn.benchmark = False
        test_dataset = Task(cfg)
        test_dataloader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.n_worker)
        
        model = Model()
        model = model.to(cfg.device)
        model.load_state_dict(t.load(model_ckpt_path, map_location=cfg.device))
        model.eval()
        
        for i, (x0, y0) in enumerate(test_dataloader):
            x0 = x0.to(cfg.device)
            y0_pred = model(x0)
            m = nn.Sigmoid()
            pred = m(y0_pred)
            for y0_, pred_ in zip(y0, pred):
                f.write(str(int(y0_.item()))+',')
                pred_ = pred_.item()
                f.write(str(pred_))
                f.write('\n')